In [1]:
# Dataset parameters
NUM_OF_USER = 10  # Number of user to be used
TEST_RATIO = 0.2 # Ratio of test data to total data
RNG_SEED = 42 # Random seed for reproducibility

# Sliding window parameters (data sampling @250 Hz)
WINDOW_SIZE = 10 * 250 # 10 seconds of data
WINDOW_STRIDE = 250 # 1 second stride

# Hyperparameters
HIDDEN_SIZE = 128
NUM_LAYERS = 2
NUM_EPOCHS = 20
LEARNING_RATE = 0.001

# Output parameters
SIGMOID_THRESHOLD = 0.8

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

def evaluate_model(y_true, y_pred):
    tp = sum((y_true == 1) & (y_pred == 1))
    tn = sum((y_true == 0) & (y_pred == 0))
    fp = sum((y_true == 0) & (y_pred == 1))
    fn = sum((y_true == 1) & (y_pred == 0))
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = 2 * (precision * recall) / (precision + recall)

    return {
        "f1": f1,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
    }


def split_train_test(df, test_size):
    # Ensure the dataframe is sorted by timestamp
    df = df.sort_values(by="n")

    # Split the data for each participant
    train_dfs = []
    test_dfs = []

    for participant_id, group in df.groupby("participant"):
        train, test = train_test_split(group, test_size=test_size, shuffle=False)
        train_dfs.append(train)
        test_dfs.append(test)

    train_df = pd.concat(train_dfs).reset_index(drop=True)
    test_df = pd.concat(test_dfs).reset_index(drop=True)

    return train_df, test_df


def create_sliding_windows(df, window_size, stride, participant_id):
    X = []
    y = []

    for pid, group in df.groupby("participant"):
        group = group.sort_values(by="n").reset_index(drop=True)
        features = group.drop(columns=["participant", "n"]).values
        label = int(pid == participant_id)

        for start in range(0, len(features) - window_size + 1, stride):
            window = features[start : start + window_size]
            X.append(window)
            y.append(label)

    return np.array(X), np.array(y)


def prepare_participant_data(
    train_df, test_df, participant_id, window_size, stride
):
    X_train, y_train = create_sliding_windows(
        train_df, window_size, stride, participant_id
    )
    X_test, y_test = create_sliding_windows(
        test_df, window_size, stride, participant_id
    )
    return X_train, X_test, y_train, y_test

In [3]:
# Load dataset
df = pd.read_parquet("dataset/gazebasevr_processed.parquet")

In [4]:
# Split the data
train_df, test_df = split_train_test(df, TEST_RATIO)

In [5]:
# Print the number of samples in the training and test set
total_samples = len(train_df) + len(test_df)
print(f"Training samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")
print(f"Ratio: train {len(train_df) / total_samples:.2f}, test {len(test_df) / total_samples:.2f}")

Training samples: 3889727
Test samples: 972455
Ratio: train 0.80, test 0.20


In [6]:
train_df.head()

,n,x,y,lx,ly,rx,ry,clx,cly,clz,...,ly_d1,rx_d1,ry_d1,x_d2,y_d2,lx_d2,ly_d2,rx_d2,ry_d2,ied
0,0.0000,0.497411,0.498659,0.502676,0.582997,0.482815,0.566561,0.456376,0.446352,0.839220,...,0.517133,0.532600,0.485751,0.132577,0.719824,0.821477,0.493740,0.526298,0.482083,0.062523
1,3.9803,0.497286,0.498659,0.502546,0.583503,0.481095,0.566643,0.456376,0.446352,0.838003,...,0.517310,0.532184,0.485807,0.132577,0.719824,0.821477,0.493740,0.526296,0.482083,0.062623
2,7.9946,0.497244,0.498626,0.503193,0.583081,0.482485,0.566915,0.449665,0.446352,0.838003,...,0.516987,0.532933,0.485934,0.132577,0.719824,0.821477,0.493738,0.526301,0.482083,0.062623
3,8.0436,0.508698,0.477689,0.511345,0.568287,0.496775,0.555513,0.463087,0.454936,0.838003,...,0.517193,0.532697,0.485725,0.132577,0.719824,0.821473,0.493740,0.526299,0.482084,0.062508
4,12.0039,0.508656,0.477689,0.511042,0.568160,0.497672,0.555952,0.463087,0.454936,0.836784,...,0.517089,0.532817,0.486050,0.132577,0.719824,0.821478,0.493739,0.526298,0.482084,0.062507


In [ ]:
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import numpy as np


# Custom Dataset class
class GazeDataset(Dataset):
    def __init__(self, data, labels):
        self.data = np.array(data)
        self.labels = np.array(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(
            self.labels[idx], dtype=torch.float32
        )


# Define the LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


: 

In [ ]:
import IPython
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
import os

np.random.seed(RNG_SEED)
participants = np.random.choice(
    df["participant"].unique(), NUM_OF_USER, replace=False
)
# participants = [201, 306, 169]
print(f"Models for participants: {participants}")

input_size = train_df.shape[1] - 2  # Exclude participant and n columns

results = []

for participant_id in participants:
    model_path = f"models/lstm/{participant_id}.pth"

    if not os.path.exists("models/lstm"):
        os.makedirs("models/lstm", exist_ok=True)

    if os.path.exists(model_path):
        print(f"Loading model for participant {participant_id} from disk")
        model = LSTMClassifier(input_size, HIDDEN_SIZE, NUM_LAYERS).to(device)
        model.load_state_dict(torch.load(model_path))
    else:
        print(f"Training model for participant {participant_id}")
        X_train, X_test, y_train, y_test = prepare_participant_data(
            train_df, test_df, participant_id, WINDOW_SIZE, WINDOW_STRIDE
        )

        train_dataset = GazeDataset(X_train, y_train)
        test_dataset = GazeDataset(X_test, y_test)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

        # Model, loss function, optimizer
        model = LSTMClassifier(input_size, HIDDEN_SIZE, NUM_LAYERS).to(device)

        weight_ratio = (len(y_train) - sum(y_train)) / sum(y_train)
        pos_weight = torch.tensor([weight_ratio]).to(device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        # Training loop
        for epoch in range(NUM_EPOCHS):
            model.train()
            for data, labels in tqdm(
                train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", leave=False
            ):
                data, labels = data.to(device), labels.to(device)
                outputs = model(data)
                loss = criterion(outputs.squeeze(1), labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            IPython.display.clear_output()
            print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.4f}")

        # Save the model
        torch.save(model.state_dict(), f"models/lstm/{participant_id}.pth")

    # Evaluation
    model.eval()
    y_true = np.array([])
    y_pred = np.array([])
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            predicted = (torch.sigmoid(outputs) > SIGMOID_THRESHOLD).long().squeeze()
            y_true = np.concatenate((y_true, labels.cpu().numpy()))
            y_pred = np.concatenate((y_pred, predicted.cpu().numpy()))

    result = evaluate_model(y_true, y_pred)
    print(result)
    results.append(result)

# Save results
if not os.path.exists("results"):
    os.makedirs("results", exist_ok=True)
results_df = pd.DataFrame(results)
results_df.to_csv("results/lstm.csv", index=False)

Models for participants: [397 431 360 389  82 445  87 117 324 388]
Training model for participant 397
